In [1]:
import http.client, urllib.parse, json, string, pickle
from time import sleep

import pandas as pd
import numpy as np

from tqdm import tqdm

np.random.seed(42)

In [2]:
def read_data(datapath):
    n_errors = 0
    was_error = False

    with open(datapath) as f:
        xy_list = list()
        tokens = list()
        tags = list()
        
        for line in f:
            # format error handling:
            if was_error:
                if line != '\n':
                    continue
                else:
                    was_error = False

            items = line.strip('\n').strip('\t').split('\t')
            if len(items) == 2:
                token, tag = items
                if token[0].isdigit():
                    tokens.append('#')
                else:
                    # is you want static noise, add it here
                    tokens.append(token.lower())
                tags.append(tag)
            elif line == '\n':
                xy_list.append((tokens, tags,))
                tokens = list()
                tags = list()
            else:
                # format error handling
                print(line)
                n_errors += 1
                was_error = True
                continue
    if n_errors > 0:
        print('Reading is done with {} errors'.format(n_errors))
    return xy_list


In [3]:
def argmax_suggestion(suggestions):
    argmax = 0
    for i, s in enumerate(suggestions):
#         print(s)
        if s['score'] > suggestions[argmax]['score']:
            argmax = i
    return argmax

def spellsafe(text, spellchecker_output):
    assert '_type' in spellchecker_output, spellchecker_output
    assert spellchecker_output['_type'] == 'SpellCheck', spellchecker_output

    result = text
    changes = {}
    for token_info in spellchecker_output['flaggedTokens']:
        
        argmax = argmax_suggestion(token_info['suggestions'])
        suggestion = token_info['suggestions'][argmax]['suggestion']
        if ' ' in suggestion or ' ' in token_info['token']:
#         if len(suggestion.split(' ')) > 1:
            print('token: ', token_info['token'], '\tsuggest: ', suggestion)
            continue
        token = token_info['token'].strip(' ').lstrip(' ')
        changes[token] = suggestion
    
    for wrong, fixed in changes.items():
        result = result.replace(wrong, fixed)
    return result

def untokenize(tokens):
    text = ''
    was_apostrophe = False
    for token in tokens:
        if token == "'":
            was_apostrophe = True
            text += token
        elif token in string.punctuation:
            was_apostrophe = False
            text += token
        elif was_apostrophe:
            was_apostrophe = False
            text += token
        else:
            was_apostrophe = False
            text += ' ' + token
    text = text.lstrip(' ')
    return text

In [69]:
text = 'Solut à tout!'

data = {'text': text}

# NOTE: Replace this example key with a valid subscription key.
key = '7559af8fffdd432da93328d18b20e85a'

host = 'api.cognitive.microsoft.com'
path = '/bing/v7.0/spellcheck?'
params = 'mkt=fr-fr&mode=spell'

headers = {'Ocp-Apim-Subscription-Key': key,
'Content-Type': 'application/x-www-form-urlencoded'}

# The headers in the following example 
# are optional but should be considered as required:
#
# X-MSEdge-ClientIP: 999.999.999.999  
# X-Search-Location: lat: +90.0000000000000;long: 00.0000000000000;re:100.000000000000
# X-MSEdge-ClientID: <Client ID from Previous Response Goes Here>

conn = http.client.HTTPSConnection(host)
body = urllib.parse.urlencode(data)
conn.request ("POST", path + params, body, headers)
response = conn.getresponse()
output = json.loads(response.read())
print(output)

{'_type': 'SpellCheck', 'flaggedTokens': [{'offset': 0, 'token': 'Solut', 'type': 'UnknownToken', 'suggestions': [{'suggestion': 'salut', 'score': 1}]}], 'correctionType': 'High'}


In [63]:
for tokens, tags in tqdm(all_data):
    data = {'text': untokenize(tokens)}
    body = urllib.parse.urlencode(data)
    conn.request ("POST", path + params, body, headers)
    response = conn.getresponse()
    output = json.loads(response.read())

    fixed_text = spellsafe(' '.join(tokens), output)
    fixed_tokens = fixed_text.split(' ')
    assert len(fixed_tokens) == len(tags), output

    fixed.append((fixed_tokens, tags))
    sleep(1/100.)

  1%|          | 2/377 [00:00<00:57,  6.57it/s]

token:  rera 	suggest:  rer a
token:  qml #uxol75 	suggest:  qmluxol75
token:  rerb 	suggest:  rer b


  1%|          | 4/377 [00:00<00:55,  6.67it/s]

token:  #rer 	suggest:  rer b
token:   stationn 	suggest:  stationner
token:   cite 	suggest:  cite u


  2%|▏         | 9/377 [00:01<00:53,  6.88it/s]

...

 98%|█████████▊| 371/377 [00:58<00:00,  6.32it/s]

token:  rerb 	suggest:  rer b
token:  rerd 	suggest:  rer d


100%|█████████▉| 376/377 [00:59<00:00,  6.34it/s]

token:  cagade 	suggest:  cage de
token:  lignej 	suggest:  ligne j


100%|██████████| 377/377 [00:59<00:00,  6.34it/s]
